# LOAD DATASET

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#LOAD DATASET
df = pd.read_csv('hasil crawling.csv')

In [ ]:
df.shape

(3867, 1)

In [ ]:
df.head(7)

,Tweet
0,kebijakan menaikkan harga bbm bersubsidi ini a...
1,harga bbm meningkat nexian messanger siap jadi...
2,setuju penyesuaian harga bbm
3,kerja kerja kerja utang utang utang mangkrak m...
4,aku setujusetuju aja kok kalo harga bbm makin ...
5,jalan tol hanya untuk bermobil utang negara pu...
6,ide goblok semua tarif dan harga selalu di bed...


# DATA PREPROCESSING

In [ ]:
#MEMBUANG ROW YANG DUPLICATE
df.drop_duplicates(subset = ['Tweet'], keep='first', inplace=True)

In [ ]:
!pip install Sastrawi
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, StopWordRemoverFactory, ArrayDictionary
from nltk.tokenize import TweetTokenizer
import string

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# MENGABAIKAN KATA2 YANG TERDAPAT DALAM STOPWORDS
stop_factory = StopWordRemoverFactory().get_stop_words()
more_stopwords = [
    'yg', 'utk', 'cuman', 'deh', 'btw', 'tapi', 'gua', 'lu', 'gue', 'gw', 'lo', 'kalo', 'trs', 'jd', 'kpn', 'kok', 'kyk', 'donk', 'yah', 'u', 'ya', 'ga', 'kamu', 'eh', 'sih', 'bang', 'br', 'kyk', 'rp', 'jt', 'kan', 'bbm', 'gpp', 'sm',
    'usah', 'mas', 'sob', 'thx', 'ato', 'jg', 'gw', 'wkwk', 'mak', 'haha', ' iy', 'k', 'dollar', 'y'
]

data = stop_factory + more_stopwords
dictionary = ArrayDictionary(data)
str = StopWordRemover(dictionary)

In [ ]:
# MENDEFINE STEMMER USING SASTRAWI
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# MELAKUKAN STEMMING UNTUK SETIAP KATA DALAM DATASET
def clean_tweets(tweet):
  tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
  tweet_tokens = tokenizer.tokenize(tweet)

  tweets_clean = []
  for word in tweet_tokens:
    if (word not in data and
        word not in string.punctuation):
      stem_word = stemmer.stem(word)
      tweets_clean.append(stem_word)

  return tweets_clean

df['Clean_Tweets'] = df['Tweet'].apply(lambda x:clean_tweets(x))
# AKAN MENGHASILKAN KATA YANG SUDAH TIDAK BERIMBUHAN DAN DIPISAH PER KATA

In [ ]:
# MENYATUKAN KEMBALI SEMUA KATA MENJADI SATU KALIMAT
def remove_punct(text):
  text = " ".join([char for char in text if char not in string.punctuation])
  return text
df['sentence'] = df['Clean_Tweets'].apply(lambda x:remove_punct(x))

In [ ]:
# MENGHILANGKAN HASIL DI KOLOM SENTENCE YANG TIDAK ADA ISINYA
df = df[df['sentence']!='']

In [ ]:
# MEMBUANG KOLOM LAIN SELAIN KOLOM "SENTENCE"
df.drop(df.columns[[0,1]], axis=1, inplace=True)

In [ ]:
# SAVE DATASET YANG SUDAH SIAP UNTUK DIKLASIFIKASIKAN
df.to_csv('data_bersih.csv', encoding='utf8', index=False)

# TRANSLATING TO ENGLISH

In [ ]:
tweet_df = pd.read_csv('data_bersih.csv')

In [ ]:
# INSTALLING GOOGLETRANS
!pip install googletrans==3.1.0a0
from googletrans import Translator

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 10.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=25d82d482af2b0edd9ac1405674018ec3a41ba9fcea6256f5b7b436f49f3d448
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
# DEFINE TRANSLATOR
translator = Translator()
translations = {}
for column in df.columns:
  unique_elements = df[column].unique()
  for element in unique_elements:
    translations[element] = translator.translate(element).text

# HASIL TRANSLATE
translations

{'bijak naik harga subsidi adalahyang laku sehat fiskal negara bijak subsidi indonesia lama salah sasar rupa risiko sistem subsidi buka semua orang beli subsidi': 'Bijag increasing the price of subsidies is what sells in a healthy fiscal country Bijag subsidies Indonesia has long mistargeted the risks of an open subsidy system, everyone buys subsidies',
 'harga tingkat nexian messanger siap jadi alternatif': 'Nexian Messenger level prices are ready to be an alternative',
 'tuju sesuai harga': 'go according to the price',
 'kerja kerja kerja utang utang utang mangkrak mangkrak mangkrak rugi rugi rugi jelas sukses danmenghasilkan naikin harga tdl pajaksemoga': 'work, work, debt, debt, debt, stall, stall, loss, loss, loss, clear success and results, increase prices before tax, hopefully',
 'aku setujusetuju aja harga makin mahal asal senyum tetep murah sama aku': 'I agree that prices are getting more expensive as long as smiles are still cheap for me',
 'jalan tol mobil utang negara lipat

In [ ]:
# MENYISAKAN SATU KOLOM AKHIR YANG BERISI HASIL TRANSLATE DARI 'TRANSLATIONS'
tweet_df.replace(translations, inplace=True)

In [ ]:
tweet_df.head()

,sentence
0,Bijag increasing the price of subsidies is wha...
1,Nexian Messenger level prices are ready to be ...
2,go according to the price
3,"work, work, debt, debt, debt, stall, stall, lo..."
4,I agree that prices are getting more expensive...


# LABELING

In [ ]:
# INSTALLING VADEDRSENTIMENT FOR AUTOMATIC LABELLING
!pip install VaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob.classifiers import NaiveBayesClassifier as NBC
import random

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# MELAKUKAN LABELING DENGAN MENGHITUNG NILAI PER ROW YANG NANTINYA DIMASUKKAN KEDALAM KOLOM BARU 'COMPOUND_SCORE
scores = [analyzer.polarity_scores(x) for x in tweet_df['sentence']]
print(scores)
tweet_df['Compound_Score'] = [x['compound'] for x in scores]

[{'neg': 0.064, 'neu': 0.854, 'pos': 0.082, 'compound': 0.1531}, {'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'compound': 0.3612}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.496, 'neu': 0.193, 'pos': 0.311, 'compound': -0.5719}, {'neg': 0.0, 'neu': 0.728, 'pos': 0.272, 'compound': 0.6808}, {'neg': 0.368, 'neu': 0.632, 'pos': 0.0, 'compound': -0.9313}, {'neg': 0.27, 'neu': 0.581, 'pos': 0.149, 'compound': -0.7003}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.25, 'neu': 0.75, 'pos': 0.0, 'compound': -0.4588}, {'neg': 0.0, 'neu': 0.597, 'pos': 0.403, 'compound': 0.4019}, {'neg': 0.075, 'neu': 0.713, 'pos': 0.212, 'compound': 0.7596}, {'neg': 0.0, 'neu': 0.349, 'pos': 0.651, 'compound': 0.6808}, {'neg': 0.214, 'neu': 0.703, 'pos': 0.084, 'compound': -0.7823}, {'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'compound': 0.4404}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.294, 'neu': 0.706, 'pos': 0.0, 'compound': -0.3612}, {'neg': 0.0, 'n

In [ ]:
# MENAMBAHKAN KOLOM BARU 'SENTIMENTS' YANG BERISIKAN NILAI AKHIR BERUPA NEGATIF / NETRAL / POSITIF
tweet_df.loc[tweet_df['Compound_Score'] < 0, 'Sentiments'] = "Negatif"
tweet_df.loc[tweet_df['Compound_Score'] == 0, 'Sentiments'] = "Netral"
tweet_df.loc[tweet_df['Compound_Score'] > 0, 'Sentiments'] = "Positif"

In [ ]:
# MEMBUANG KOLOM 'COMPOUND_SCORE' KARENA YANG DIBUTUHKAN NAIVE BAYES HANYALAH 'SENTENCE' DAN 'SENTIMENTS'
tweet_df.drop(tweet_df.columns[[1]], axis=1, inplace=True)

In [ ]:
# MENGUBAH MENJADI BENTUK LIST KARENA NAIVE BAYES HANYA MENERIMA BENTUK LIST
dataset = [tuple(x) for x in tweet_df.to_records(index=False)]

In [ ]:
# MEMBUAT SET BARU UNTUK MENAMPUNG NILAI DARI MASING-MASING POSITIF NEGATIF DAN NETRAL
set_positif = []
set_negatif = []
set_netral = []

# PENGULANGAN DALAM DATASET, JIKA POSITIF AKAN DIMASUKKAN KEDALAM SET_POSITIF, DST
for n in dataset:
    if(n[1] == 'Positif'):
        set_positif.append(n)
    elif(n[1] == 'Negatif'):
        set_negatif.append(n)
    else:
        set_netral.append(n)

# RANDOM SUBSUMPLING
set_positif = random.sample(set_positif, k=int(len(set_positif)/2))
set_negatif = random.sample(set_negatif, k=int(len(set_negatif)/2))
set_netral = random.sample(set_netral, k=int(len(set_netral)/2))

# TRAINING DATA GABUNGAN DARI SET YANG SUDAH DIBUAT SEBELUMNYA
train = set_positif + set_negatif + set_netral

# SET BARU UNTUK MENAMPUNG DATA TRAIN YANG AKAN DIGUNAKAN
train_set = []

for n in train:
    train_set.append(n)

In [ ]:
# PROSES TRAINING
cl = NBC(train_set)

In [ ]:
# NILAI AKURASI
print('Akurasi Test: ', cl.accuracy(dataset))

Akurasi Test:  0.7794155676234807


# ANALISA DATA

In [ ]:
from sklearn import metrics
from textblob import TextBlob

In [ ]:
data_tweet = list(tweet_df['sentence'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0
for i, tweet in enumerate(data_tweet):
  analysis = TextBlob(tweet, classifier=cl)

  if analysis.classify() == 'Positif':
    total_positif +=1
  elif analysis.classify() == 'Negatif':
    total_negatif +=1
  else:
    total_netral +=1

  status.append(analysis.classify())
  total +=1

print(f'\nHasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data: {total}')


Hasil Analisis Data:
Positif = 2239
Netral = 1008
Negatif = 620

Total Data: 3867


In [ ]:
# MEMBUAT DATAFRAME BARU YANG BERISI HASIL DARI ANALISIS DATA
sentiment_df = pd.DataFrame({'Bayes': status})

In [ ]:
# MENGGABUNGKAN HASIL ANALISIS DATA DENGAN DATASET YANG SUDAH ADA LABELNYA
df = pd.concat([tweet_df, sentiment_df], axis=1)

In [ ]:
# MENYIMPAN HASIL PERBANDINGAN
df.to_csv('hasilperbandingan.csv', encoding='utf8', index=False)

In [ ]:
# METRIKS
print(metrics.classification_report(df['Sentiments'], df['Bayes']))

              precision    recall  f1-score   support

     Negatif       0.79      0.54      0.64       918
      Netral       0.70      0.81      0.75       878
     Positif       0.81      0.88      0.84      2071

    accuracy                           0.78      3867
   macro avg       0.77      0.74      0.74      3867
weighted avg       0.78      0.78      0.77      3867



# KESIMPULAN

AKURASI YANG DIDAPATKAN MENCAPAI ANGKA 78% <br>
Dengan total data sekitar 3867
<br>
<br>
DENGAN PERBANDINGAN <br>
Sentiment Positif berjumlah 2239<br>
Sentiment Netral berjumlah 1008<br>
Sentiment Negatif berjumlah 620<br>
